# Beta Model!

- This current working model was only trained to 1 epoch. This file will reflect the attempt made to train it up to 50 epochs ... which was a mistake as the google colab server timed out, despite having google colab +. 

- The model was trained and compiled on a TPU in google colab. 

- The working model and all its outputs are available in the repo. **Links are provided in this file within their respective sections of the code.**

- The next step is to build a web integration to deploy the model AND to integrate image augmentation techniques. For this week, the focus is to complete the file to deploy the model. 


### PLEASE HELP ME FIGURE OUT THE IMAGE AUGMENTATION SECTION. I have included links to the tutorials I have tried to follow as well as its resources. 

### Import Dependencies And Define Paths

In [1]:
import numpy as np 
import pandas as pd  
# %matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.metrics import roc_curve, auc
import keras
import tensorflow as tf

In [2]:
# Mount your google drive to access the files 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import Data Files 

rtrain =pd.read_csv("/content/drive/MyDrive/NIH-CXR/train_set.csv")
rtest = pd.read_csv("/content/drive/MyDrive/NIH-CXR/test_set.csv")
rval = pd.read_csv("/content/drive/MyDrive/NIH-CXR/val_set.csv")

### Clean the Dataframes and extract labels 

In [5]:
# Fixes the paths to locate files in my google drive

def path_fix(df, column, header):
  copy_df = df.copy()
  # Insert the header path for your drive folder
  copy_df[column] = header + copy_df[column].astype(str)
  # Replace the backslashes with forward slashes as a string. \\\\ is for regex purposes
  copy_df[column] = copy_df[column].str.replace("\\\\", "/")
    
  return copy_df

In [6]:
# Function to clean the sets and extract the labels. 

def label_cleaner(raw_df):
    # Copy the Df into two separate dataframes, one for saving as csv and one for output
    copy_df = raw_df.copy()

    # Create a column for categorical labels list in case of use
    copy_df["Labels"] = copy_df["Finding Labels"].replace("No Finding", "").str.split("|", expand = False)
    # Get a list of all diseases 
    di = list(chain(*copy_df["Finding Labels"].replace("No Finding", "").str.split("|", expand = False).tolist()))
    # Drop unnecessary columns from df 
    copy_df.drop(columns = ['Unnamed: 0', 'og_idx'], axis= 1, inplace= True )

    #Extract the list of diseases as a regualr list and as an numpy array because the model likes arrays    
    ls = np.unique(di).tolist()
    ls.remove('')
    np_ls = np.array(ls)

    return copy_df, ls, np_ls


In [7]:
# Define constants for the column name and the path header. 

COL = "path"
HEAD = "/content/drive/MyDrive/NIH-CXR/"

In [8]:
# Fix the paths specific to google drive format 

ftrain = path_fix(rtrain, COL, HEAD)
ftest = path_fix(rtest, COL, HEAD)
fval = path_fix(rval, COL, HEAD)

ftrain.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Unnamed: 0,og_idx,Image Index,Finding Labels,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,29453,44578,00011460_052.png,No Finding,/content/drive/MyDrive/NIH-CXR/images_005/imag...,0,0,0,0,0,0,0,0,0,0,0,0,0
1,33813,50949,00012880_011.png,No Finding,/content/drive/MyDrive/NIH-CXR/images_006/imag...,0,0,0,0,0,0,0,0,0,0,0,0,0
2,32929,49712,00012620_014.png,Effusion,/content/drive/MyDrive/NIH-CXR/images_006/imag...,0,0,0,0,1,0,0,0,0,0,0,0,0
3,11713,17922,00004822_026.png,Atelectasis|Infiltration,/content/drive/MyDrive/NIH-CXR/images_003/imag...,1,0,0,0,0,0,0,1,0,0,0,0,0
4,76383,111882,00030650_007.png,Effusion|Infiltration|Nodule|Pleural_Thickening,/content/drive/MyDrive/NIH-CXR/images_012/imag...,0,0,0,0,1,0,0,1,0,1,1,0,0


In [ ]:
# Clean and extract labels

train_set, labels, np_labels = label_cleaner(ftrain)
test_set, _, _ = label_cleaner(ftest)
val_set, _, _ = label_cleaner(fval)

print(f"Train Set Shape: {train_set.shape},\nTest Set Shape: {test_set.shape},\nVal Set Shape: {val_set.shape}")

Train Set Shape: (30000, 17),
Test Set Shape: (10000, 17),
Val Set Shape: (10000, 17)


In [10]:
labels

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

### In case We need to pre-process our images external to the generator, this will split the X and Y from all the sets.

- This section would not need to be included if we are using the `ImageDataGenerator()` and the `flow_from_dataframe()` generator functions to generate batches of images. 


In [10]:
# Function to split between x and y 

def xysplitter(df, xcol, ycol): 
  copy_df = df.copy()
  x = copy_df[xcol]
  y = copy_df[ycol]

  return x, y


In [11]:
# Split the sets

xtrain, ytrain = xysplitter(train_set, xcol = 'path', ycol = labels)
xtest, ytest =  xysplitter(test_set, xcol = 'path', ycol = labels)
xval, yval =  xysplitter(val_set, xcol = 'path', ycol = labels)

### Sample Histogram Equalization (PLEASE HELP ME)

- Please send help. Below are two tutorial links that I have tried to follow but to no avail. 

    - [Hist Eq integration to ImageDataGenerator()](https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085)

    - [Histogram Equalize RGB images external to Keras](https://towardsdatascience.com/histogram-equalization-a-simple-way-to-improve-the-contrast-of-your-image-bcd66596d815). Current Approach

- Additionally I have found other modules' documentation that could help me perform the augmentation after integration into my current working code. 

    - [Kornia Module](https://kornia-tutorials.readthedocs.io/en/latest/image_histogram.html)

    - [Tensor Flow add-ons](https://www.tensorflow.org/addons/api_docs/python/tfa/image/equalize)

    - [PyTorch](https://pytorch.org/vision/main/generated/torchvision.transforms.functional.equalize.html)

    - [PyTorch Transofrmations: Including Gaussian Blur](https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py). Honestly, I'm probably going to settle for this? 


In [ ]:
# Sample an image from the train set
# img = xtrain[0]

In [ ]:
# Import Dependencies 
import cv2 as cv

# Function to histogram equalize images across RGB channels. The source code is the second link up top.
def histogram_equalization(img_in):
# segregate color streams
  b,g,r = cv.split(img_in)
  h_b, bin_b = np.histogram(b.flatten(), 256, [0, 256])
  h_g, bin_g = np.histogram(g.flatten(), 256, [0, 256])
  h_r, bin_r = np.histogram(r.flatten(), 256, [0, 256])
# calculate cdf    
  cdf_b = np.cumsum(h_b)  
  cdf_g = np.cumsum(h_g)
  cdf_r = np.cumsum(h_r)
  
# mask all pixels with value=0 and replace it with mean of the pixel values 
  cdf_m_b = np.ma.masked_equal(cdf_b,0)
  cdf_m_b = (cdf_m_b - cdf_m_b.min())*255/(cdf_m_b.max()-cdf_m_b.min())
  cdf_final_b = np.ma.filled(cdf_m_b,0).astype('uint8')

  cdf_m_g = np.ma.masked_equal(cdf_g,0)
  cdf_m_g = (cdf_m_g - cdf_m_g.min())*255/(cdf_m_g.max()-cdf_m_g.min())
  cdf_final_g = np.ma.filled(cdf_m_g,0).astype('uint8')

  cdf_m_r = np.ma.masked_equal(cdf_r,0)
  cdf_m_r = (cdf_m_r - cdf_m_r.min())*255/(cdf_m_r.max()-cdf_m_r.min())
  cdf_final_r = np.ma.filled(cdf_m_r,0).astype('uint8')
# merge the images in the three channels
  img_b = cdf_final_b[b]
  img_g = cdf_final_g[g]
  img_r = cdf_final_r[r]

  img_out = cv.merge((img_b, img_g, img_r))
  
  return img_out


In [15]:
# imgin = cv.imread(img)
# out = histogram_equalization(imgin)
# out.dtype

In [16]:
# image_decoded = tf.convert_to_tensor(out)

In [17]:
# # Read an image from a file
# image_string = tf.io.read_file(img)
# # Decode it into a dense vector
# image_decoded = tf.image.decode_png(image_string, channels=3)

# image_decoded.dtype

### Image Batching: Generator Version is Commented out

- Image data is preprocessed and fed into the model in batches by iterating through the directories where the images are located. 

- **Current version creates a custom generator that optimizes the batching process.** The source code is located [here](https://towardsdatascience.com/multi-label-image-classification-in-tensorflow-2-0-7d4cf8a4bc72)

- The Generator version (not current) was obtained from [here](https://vijayabhaskar96.medium.com/multi-label-image-classification-tutorial-with-keras-imagedatagenerator-cd541f8eaf24)

- The Generator version is a lot more memory intensive and slower. ***The current version does not include any augmentations.***

- Based on the [literature](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0265949), the optimal augmentation methods are [Histogram Equalization, Guassian Blur, and (possibly)Adaptive Masking]. 


In [18]:
# Define Constants for image parameters. 

IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

In [19]:
# Function to read the image files and transform them into vectors

def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_png(image_string, channels=CHANNELS)
    
    """# Read Image using the cv module 
    image = cv.imread(str(filename))
    # Perform Histogram equalization across all 3 channels. 
    equalized = histogram_equalization(image)
    # convert to tensors from nparray
    image_decoded = tf.convert_to_tensor(equalized)
    """
    
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [20]:
# Define constants for the batching and shuffling 

BATCH_SIZE = 32 # Big enough to not crash the processor
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 256 # Shuffle the training data by a chunck of 256 observations

In [21]:
# Function to Generate the dataset required

def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
# Create the DataSets

train_ds = create_dataset(xtrain, ytrain)
val_ds = create_dataset(xval, yval)

In [23]:
# Define image size and instantiate an Image Data Generator and define its parameters
# The parameters focus on random modifications to the images to ensure the model doesn't focus on the wrong features
# Since the image is in greyscale we don't have to worry about messing with the parameters that deal with color/ brightness
# Just the physical modifications of the images themselves. 
# We have a lot of leeway with the images since the features should be localized towards the middle most of the time.
# Don't go too wild with the modifications and shift out of range/ focus of the features.
"""
image_size = (224, 224)

imdg = ImageDataGenerator(
    rescale = 1/255,
    samplewise_center = True, # Set each sample mean to 0.  
    samplewise_std_normalization = True, # Divide each input by its std. 
    horizontal_flip = True, # Flips samples horizontally randomly to prevent over fitting over wrong features
    vertical_flip = False, 
    width_shift_range=0.1, # Define how the images should be randomly shifted when fed, 10% of image 
    height_shift_range= 0.05, # Define how the images should be  randomly shifted when fed, 5% of image
    shear_range = 0.05, # slants the images in a certain degree based on the value.
    zoom_range=0.05, # magnify the images randomly by 5% 
    rotation_range= 2, # rotate the images by 2 
fill_mode = 'reflect' # if images have empty values due to modifications, reflect the image. 
)
"""

In [24]:
# Define the generators for each of the dataframes
"""
train_generator = imdg.flow_from_dataframe(
    dataframe=train_set,
    x_col="path",
    y_col= np_labels,
    target_size=image_size,
    batch_size=32,
    class_mode='raw',
    shuffle=True,
    seed=10,
)
val_generator = imdg.flow_from_dataframe(
    dataframe=val_set,
    x_col="path",
    y_col= np_labels,
    target_size=image_size,
    batch_size= 256,
    class_mode='raw', 
    shuffle=False,
    seed=10,
)

test_generator = imdg.flow_from_dataframe(
    dataframe=test_set,
    x_col="path",
    target_size=image_size,
    batch_size= 200,
    class_mode= None,
    shuffle=False,
    seed=10,
)
"""

In [ ]:
# Sample the images to be fed from the training set and visuzalize them
# This code is currently not being deployed since the current batching method does not use the generator. 
# Retrofitting required for this code to be viable with the current set up 
"""
# Adjust the ImageData Generator accordingly based on output variability. 

plt.style.use('dark_background')
t_x, t_y = next(train_generator)
# instantiate a figure and axes to plug images into 
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
# Define how each of the images will be plotted each image will have its label associated so importnat to zip
# Flatten the images to reduce dimensionality
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    # Display the image using imshow along a new axies instance across all dimensions 
    # Define the color map for bones and xrays using cmap and let it cover over 150% using vmin/vmax 
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)
    # Loop through the labels in the target output using the labels and plot them accordingly. 
    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(labels, c_y) 
                             if n_score>0.5]))
    # Turn off the axies ticks so that the images aren't cluttered. 
    c_ax.axis('off')
"""

"\n# Sample the images to be fed from the training set and visuzalize them\n# Adjust the ImageData Generator accordingly based on output variability. \n\nplt.style.use('dark_background')\nt_x, t_y = next(train_generator)\n# instantiate a figure and axes to plug images into \nfig, m_axs = plt.subplots(4, 4, figsize = (16, 16))\n# Define how each of the images will be plotted each image will have its label associated so importnat to zip\n# Flatten the images to reduce dimensionality\nfor (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):\n    # Display the image using imshow along a new axies instance across all dimensions \n    # Define the color map for bones and xrays using cmap and let it cover over 150% using vmin/vmax \n    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)\n    # Loop through the labels in the target output using the labels and plot them accordingly. \n    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(labels, c_y) \n                  

### Train the Model

- The model is based off the [literature](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0265949), previously discussed. 

- The base feature extractor layer is [ResNet101](https://www.kaggle.com/datasets/pytorch/resnet101) that includes all the convolution and max pooling layers. Transfer Learning can be implemented to also train this layer to extract the most optimal features but it is already pre-trained so it is unnecessary. 

- The model was last run for 12 hours to attempt to reach 50 epochs but crashed due to google colab server timeout. This means we should only train up to 20 epochs max. Plus it did not seem like the val loss was improving anyways after about 20 epochs. 

- The working model that is currently uploaded to the repo as [`beta_model.h5`](https://github.com/emilybstevens/CXR-ML/blob/main/b_model.h5) was only trained for 1 epoch. 


In [26]:
# Import Dependencies for the CNN

from keras.applications.resnet import ResNet101
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint

In [27]:
# Differential Model Incorporating ResNet101

feature_extractor_layer = ResNet101(input_shape =  (IMG_SIZE,IMG_SIZE,CHANNELS), 
                                 include_top = False, weights = None, pooling = max)

# trainable has to be false in order to freeze the layers
feature_extractor_layer.trainable = False # 
b_model = Sequential()
b_model.add(feature_extractor_layer)
b_model.add(GlobalAveragePooling2D())
b_model.add(Flatten()) # Flatten out the tensor so that the drop out layer is not processing too much information. 
b_model.add(Dropout(0.2))
b_model.add(Dense(512, activation= 'relu', name='hidden_layer'))
b_model.add(Dropout(0.5))
b_model.add(Dense(13, activation = 'sigmoid', name='output_layer'))
b_model.compile(Adam(learning_rate=1e-4),loss="binary_crossentropy",metrics=['binary_accuracy', 'mae'])
b_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 hidden_layer (Dense)        (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                        

In [28]:
# Save checkpoints

weights = "weights/{}_weights.best/hdf5".format("modelbeta")

checkpoint = ModelCheckpoint(
    weights, monitor="val_loss", 
    verbose = 50, 
    save_best_only = True,
    mode = "min", 
    save_weights_only = True  
)


In [ ]:
# Train the model 
b_model.fit(train_ds,
  epochs=1,
  validation_data = val_ds, 
  callbacks = checkpoint)

Epoch 1/50
938/938 [==============================] - ETA: 0s - loss: 0.2715 - binary_accuracy: 0.9172 - mae: 0.1531
Epoch 1: val_loss improved from inf to 0.25639, saving model to weights/modelbeta_weights.best/hdf5
938/938 [==============================] - 1073s 1s/step - loss: 0.2715 - binary_accuracy: 0.9172 - mae: 0.1531 - val_loss: 0.2564 - val_binary_accuracy: 0.9188 - val_mae: 0.1369
Epoch 2/50
938/938 [==============================] - ETA: 0s - loss: 0.2614 - binary_accuracy: 0.9187 - mae: 0.1435
Epoch 2: val_loss improved from 0.25639 to 0.25598, saving model to weights/modelbeta_weights.best/hdf5
938/938 [==============================] - 1088s 1s/step - loss: 0.2614 - binary_accuracy: 0.9187 - mae: 0.1435 - val_loss: 0.2560 - val_binary_accuracy: 0.9188 - val_mae: 0.1390
Epoch 3/50
938/938 [==============================] - ETA: 0s - loss: 0.2602 - binary_accuracy: 0.9188 - mae: 0.1430
Epoch 3: val_loss improved from 0.25598 to 0.25571, saving model to weights/modelbeta_w

In [ ]:
# Fit and train use fit_generator() to fit in the batches of images 

"""
b_model.fit_generator(
    train_generator, 
    steps_per_epoch = train_generator.n//train_generator.batch_size, 
    validation_data = val_generator,
    validation_steps = val_generator.n//val_generator.batch_size,
    epochs = 1,
    callbacks = checkpoint 
)
"""

### Model Predictions

- The model was tested on 10,000 images separate from the train and val sets. 

- ImageDataGenerator was used here to generate the batches of images for the testing set as previous method requires lables to be present. I have not refactored the code to be suitable for the test set. 

- The model outputs probailities across different labels. 

- Single image predictions will be performed to assess classifications and for further deployment to the web app. This will be done in a separate file. 

See the Avg Distribution of labels in the current test_set

In [ ]:
# Transform the labels into a list of binaries
test_Y = test_set.apply(lambda x: [x[labels].values], axis = 1).map(lambda x: x[0])

# See the average distribution of the true labels in the test set 
for c_label, s_count in zip(labels, 100*np.mean(test_Y,0)):
    print('%s: %2.2f%%' % (c_label, s_count))

Predict using Test set

In [ ]:
# Instantiate ImageDataGenerator instance only for the test data
imdg = ImageDataGenerator(rescale = 1/255)

# Create a generator for the test images since the custom function requies labels 

test_generator = imdg.flow_from_dataframe(
    dataframe=test_set,
    x_col="path",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=25,
    class_mode=None,
    shuffle=False
)

# Predict the Test Dataset 
pred_y = b_model.predict(
    test_generator, 
    steps = test_generator.n//test_generator.batch_size,
    verbose=1)

In [ ]:
# Transform the binarized list of labels to a numpy array or else a for loop will not be able to index well
nptest_Y = test_set[labels].to_numpy(dtype = int)
nptest_Y.shape

### Model Evaluation and Visualizations

- The predictions were visually evaluated using the AUC/ROC curve and confusion matrices for each individual labels. Both of these visualizations evaluate the differences between the model's predictions of True/False Positives and True/False Negatives. 

- The [AUC/ROC curve](https://github.com/emilybstevens/CXR-ML/blob/main/Resources/betanet/beta_1epoch.png) for the current working model is availabel in the repo. 
    
    - Higher AUC, the better the model is at predicting a specific class label. The ROC Curve displays the model's classification accuracy across each label based on the true positives and false positives. Ideally, the curves for all labels should be veering towards the top left corner since we want to maximize the true positive and minimize the false positive values. 

- Each label's [confusion matrices](https://github.com/emilybstevens/CXR-ML/tree/main/Resources/betanet) are available as png images beginning with `cm`. 

- Based on the performance of the current model, it is evident that image augmentation plays a very important role in the accuracy of the model. The confusion matrices indicate that the model is capable of predicting true negatives well but not true positives. Some labels do better than others. This could mean:
    
    - The features are not being picked out well by the extractor layer due to lack of augmentation. 
    - The model should only be trained up to 10 epochs or there is a risk of overtraining for feature identification. 
    - The sample size distribution should include more no_finding images to the model can generalize the classifications better. 

AUC/ROC Curve

In [ ]:
# Create an AUC / ROC curve 
fig, c_ax = plt.subplots(1,1, figsize = (15, 15))

# Loop through all the labels and get the index and values. 
for (idx, c_label) in enumerate(labels):
    # create variables for the roc curve based on its output.
    #The for loop loops through all the values in the list of binarized labels based on index via column of the labels
    fpr, tpr, thresholds = roc_curve(nptest_Y[:,idx].astype(int), pred_y[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('beta_50epochs.png')

Save the outputs that are greater than 20% probability as a csv file of predicted labels 

In [ ]:
# See Predictions greater than 20% for the provisional model and save to csv
predictions = (pred_y > 0.2).astype(int)
columns=labels
#columns should be the same order of y_col
results=pd.DataFrame(predictions, columns=columns)
results["Filenames"]=test_set["Image Index"].tolist()
ordered_cols=["Filenames"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv('beta_net_predictions.csv')

print(results)

Confusion Matrices

In [ ]:
# Import Dependencies 
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

# Create the confusion matrix for each of the labels 
conf_mat = multilabel_confusion_matrix(y_true = nptest_Y, y_pred= np.array(results[labels], dtype = int))

# Plot the individual confusion matrices for the test sample. 

for i in range(len(conf_mat)):
  disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat[i])
  disp.plot()
  plt.title(labels[i])
  plt.savefig(f"cm_{labels[i]}")
  plt.show()

### Save the Model 

In [ ]:
# Required to save models in HDF5 format
!pip install pyyaml h5py

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
b_model.save('b_model.h5') 

Please feel free to write me comments ici:


